# Week #1
## Знакомство с инструментами
### Предобработка данных в Pandas

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('data/titanic.csv', index_col = 'PassengerId')
data.head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print 'Какое количество мужчин и женщин ехало на корабле?'
# В качестве ответа приведите два числа через пробел.
print ' '.join(map(str, data.Sex.value_counts().get_values()))

Какое количество мужчин и женщин ехало на корабле?
577 314


In [5]:
print 'Какой части пассажиров удалось выжить?'
# Посчитайте долю выживших пассажиров. Ответ приведите в процентах 
# (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.
print round(float(np.nonzero(data.Survived != 0)[0].size) / data.shape[0] * 100, 2)

Какой части пассажиров удалось выжить?
38.38


In [6]:
print 'Какую долю пассажиры первого класса составляли среди всех пассажиров?'
# Ответ приведите в процентах (число в интервале от 0 до 100, знак процента не нужен), 
# округлив до двух знаков.
print round(float(np.nonzero(data.Pclass == 1)[0].size) / data.shape[0] * 100, 2)

Какую долю пассажиры первого класса составляли среди всех пассажиров?
24.24


In [7]:
print 'Какого возраста были пассажиры?'
# Посчитайте среднее и медиану возраста пассажиров. 
# В качестве ответа приведите два числа через пробел.
print np.nanmean(data.Age), np.nanmedian(data.Age)

Какого возраста были пассажиры?
29.6991176471 28.0


In [8]:
print 'Коррелируют ли число братьев/сестер/супругов с числом родителей/детей?'
# Посчитайте корреляцию Пирсона между признаками SibSp и Parch.
print np.corrcoef(data.SibSp, data.Parch)[0, 1]

Коррелируют ли число братьев/сестер/супругов с числом родителей/детей?
0.41483769862


In [9]:
print 'Какое самое популярное женское имя на корабле?'
# Извлеките из полного имени пассажира (колонка Name) его личное имя (First Name). 
# Это задание — типичный пример того, с чем сталкивается специалист по анализу данных. 
# Данные очень разнородные и шумные, но из них требуется извлечь необходимую информацию. 
# Попробуйте вручную разобрать несколько значений столбца Name и выработать правило 
# для извлечения имен, а также разделения их на женские и мужские.
mans = data.Sex == 'male'
womems = data.Sex == 'female'
miss = data.Name.str.contains('Miss.')
mrs = data.Name.str.contains('Mrs.')
data.ix[mans, 'FName'] = data[mans]['Name'].str.extract('\. ([^().,]*)', expand=False)
data.ix[miss, 'FName'] = data[miss]['Name'].str.extract('\. ([^().,]*)', expand=False)
data.ix[mrs, 'FName'] = data[mrs]['Name'].str.extract('\(([^().,]*)', expand=False)
data[data.Sex == 'female']['FName'].value_counts()

Какое самое популярное женское имя на корабле?


Mary                          7
Bertha                        4
Anna                          4
Anna Sofia                    3
Katherine "Katie"             2
Lucile Polk                   2
Elizabeth                     2
Alice                         2
Ellen "Nellie"                2
Marija                        2
Kate                          2
Hanora "Nora"                 2
Imanita Parrish Hall          1
Anna Sigrid Maria Brogren     1
Edith Martha Bowerman         1
Alice Elizabeth               1
Johanna Persdotter Larsson    1
Ingeborg Constanzia           1
Mary Conover                  1
Albina                        1
Asuncion                      1
Mathilde                      1
Brigdet Delia                 1
Georgette Alexandra           1
Susan Parker "Suzette"        1
Ellen                         1
Amy Frances Christy           1
Anna Sophia Atkinson          1
Marie Grice                   1
Sylvia Mae Harbaugh           1
                             ..
Marjorie

## Решающие деревья
### Важность признаков

In [10]:
# Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), 
# возраст пассажира (Age) и его пол (Sex).
# Обратите внимание, что признак Sex имеет строковые значения.
# Выделите целевую переменную — она записана в столбце Survived.
# В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен 
# их возраст. Такие записи при чтении их в pandas принимают значение nan. 
# Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.
data.ix[mans, 'Sex'] = 0
data.ix[womems, 'Sex'] = 1
data1 = data[['Survived', 'Pclass', 'Fare', 'Age', 'Sex']].dropna()
X = data1[['Pclass', 'Fare', 'Age', 'Sex']]
y = data1['Survived']

# Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию.
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=241)
clf.fit(X, y)

# Вычислите важности признаков и найдите два признака с наибольшей важностью. 
# Их названия будут ответами для данной задачи (в качестве ответа укажите названия 
# признаков через запятую или пробел, порядок не важен).
importances = clf.feature_importances_
feature_importances = np.vstack((X.columns, importances))
feature_importances

array([['Pclass', 'Fare', 'Age', 'Sex'],
       [0.14751816099515025, 0.2953846784065746, 0.25658494964003575,
        0.3005122109582393]], dtype=object)